## Pandas III: Datos categóricos y temporales

<a id="indice"></a>
# Índice


* [1. Datos categóricos](#categoricos) 
  * [Categorías con orden](#categoricos-orden) 
  * [Manipulación de las categorías](#categoricos-manipulacion)
* [2. Datos temporales](#temporales) 
  * [Timestamp](#temporales-timestamp)
    * [Localizando los objetos timezone](#temporales-localize)
    * [Atributos y funciones](#temporales-atributos)
    * [Timestamps en series](#temporales-series)
  * [DatetimeIndex](#temporales-datetimeindex)
    * [Creación de DatetimeIndex](#temporales-datetimeindex-creacion)
  * [Period](#temporales-period)
  * [PeriodIndex](#temporales-PeriodIndex)
  * [Función `resample`](#temporales-resample)

# DataFrame

En esta libreta vamos a trabajar con dos `DataFrames`. El primero, "MQTT-Events.csv", contiene datos de monitorización de una red en la que se genera tráfico MQTT. En el segundo, "packetLengthPrediction.csv", disponemos datos de predicción del tamaño de los paquetes en función del resto de columnas en el primer `DataFrame`. Los datos fueron tomados el día 3 de febrero de 2018, desde las 9PM hasta las 11PM.

In [1]:
import pandas as pd

df_events = pd.read_csv("../data/MQTT-events.csv")
df_predictions = pd.read_csv("../data/packetLengthPrediction.csv")

In [2]:
df_events.head()

,timestamp,packetLength,info,networkInterface.sourceMAC,networkInterface.destinationMAC,internet.sourceIP,internet.destinationIP,internet.protocol,TCP.sourcePort,TCP.destinationPort,TCP.flags,TCP.calculatedWindowSize,UDP.sourcePort,UDP.destinationPort,MQTT.flags,MQTT.message,MQTT.topic,MQTT.messageLength,MQTT.frameCounter
0,1517688352799319,100,Publish Message,08:00:27:71:f1:05,08:00:27:53:41:85,192.168.1.7,192.168.1.5,MQTT,52588,1883,0x018,229,-1,-1,0x30,62.7216410003,fakeSensor2/sound,32,1
1,1517688352837583,66,1883 > 52588 [ACK] Seq=1 Ack=35 Win=227 Len=...,08:00:27:53:41:85,08:00:27:71:f1:05,192.168.1.5,192.168.1.7,TCP,1883,52588,0x010,227,-1,-1,-1,-1,-1,-1,1
2,1517688352965108,100,Publish Message,08:00:27:0e:06:81,08:00:27:53:41:85,192.168.1.4,192.168.1.5,MQTT,59662,1883,0x018,229,-1,-1,0x30,46.1680763098,fakeSensor3/sound,32,1
3,1517688352965147,66,1883 > 59662 [ACK] Seq=1 Ack=35 Win=227 Len=...,08:00:27:53:41:85,08:00:27:0e:06:81,192.168.1.5,192.168.1.4,TCP,1883,59662,0x010,227,-1,-1,-1,-1,-1,-1,1
4,1517688354027486,100,Publish Message,08:00:27:0e:06:81,08:00:27:53:41:85,192.168.1.4,192.168.1.5,MQTT,59662,1883,0x018,229,-1,-1,0x30,31.8849343769,fakeSensor3/sound,32,1


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#000000"></i></font></a></div>
<a id="categoricos"></a>

# 1. Datos categóricos

_Pandas_ implementa un tipo de datos específico para el trabajo con datos categóricos, es decir, variables que pueden tomar un número finito de valores. Aunque éstos se visualizan generalmente como Strings, internamente se representan mediante enteros, lo cual permite tratarlos eficientemente.

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#000000"></i></font></a></div>

## Objetos `pd.Categorical`

El objeto `pd.Categorical` permite representar **una colección** (_no una serie_) de datos relativos a categorías. Por defecto, considera tantas categorías como detecta en los datos, pero éstas se pueden especificar mediante el argumento `categories`.

In [3]:
log_levels = pd.Categorical(["DEBUG","DEBUG","INFO","DEBUG", "ERROR"])
print(log_levels)

print("\n")

# Solo considera tres categorías. 
log_levels = pd.Categorical(["DEBUG","DEBUG","INFO","DEBUG", "ERROR"],
                           categories=["DEBUG","INFO", "WARNING","ERROR", "CRITICAL"])
print(log_levels)

['DEBUG', 'DEBUG', 'INFO', 'DEBUG', 'ERROR']
Categories (3, object): ['DEBUG', 'ERROR', 'INFO']


['DEBUG', 'DEBUG', 'INFO', 'DEBUG', 'ERROR']
Categories (5, object): ['DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL']


<div class="alert alert-block alert-info">

<i class="fa fa-info-circle" aria-hidden="true"></i> En una colección de valores categóricos, aquellos cuya categoría no ha sido especificada se codifican como `np.NaN`.
</div>

In [4]:
log_levels = pd.Categorical(["DEBUG","DEBUG","INFO","DEBUG", "ERROR"],
                           categories=["DEBUG","INFO"])
print(log_levels)

['DEBUG', 'DEBUG', 'INFO', 'DEBUG', NaN]
Categories (2, object): ['DEBUG', 'INFO']


<div class="alert alert-block alert-info">

<i class="fa fa-info-circle" aria-hidden="true"></i> Si disponemos de una serie creada, podemos acceder a las categorías, al igual que en los `DataFrames` accedemos a `columns` o `index`, como `log_levels.categories`.
</div>

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#000000"></i></font></a></div>
<a id="categoricos-orden"></a>

## Categorías con orden

Cuando se comparan `Series` numéricas, se aplica la comparación entre los valores numéricos. Cuando se comparan `Series` que son cadenas de texto, se aplica la comparación entre los strings (por órden alfabético).

Cuando se comparan `Series` categoricas, por defecto no existe un órden, por lo que la comparación causaría una **excepción** (_solo se permitiría la comparación `==`_). Existe un argumento `ordered=True` que se especifica en la creación del objeto `pd.Category`, y sirve paa indicar que la lista de categorías tiene el orden especificado. En este caso, sí se podría comparar y se seguiría este orden de las categorías.

In [5]:
# Fallaría la comparación
log_levels_jan = pd.Categorical(["debug","debug","info","info", "error", "debug"],
                          categories=["debug","info", "warning","error", "critical"])
log_levels_feb = pd.Categorical(["debug","info","debug","warning", "debug", "info"],
                          categories=["debug", "info", "warning","error", "critical"])

# Causa una excepción
# log_levels_jan < log_levels_feb

In [6]:
log_levels_jan = pd.Categorical(["debug","debug","info","info", "error", "debug"],
                          categories=["debug","info", "warning","error", "critical"], ordered=True)
log_levels_feb = pd.Categorical(["error","info","debug","warning", "debug", "info"],
                          categories=["debug","info", "warning","error", "critical"], ordered=True)

log_levels_jan < log_levels_feb

array([ True,  True, False,  True, False,  True])

### Reordenación de categorías

También es posible cambiar solamente el orden entre las categorías mediante `reorder_categories()`.

In [7]:
log_levels_feb = log_levels_feb.reorder_categories(["critical","error","warning", "info", "debug"], ordered=False)
log_levels_feb

['error', 'info', 'debug', 'warning', 'debug', 'info']
Categories (5, object): ['critical', 'error', 'warning', 'info', 'debug']

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#000000"></i></font></a></div>
<a id="categoricos-manipulacion"></a>

## Manipulación de las categorías

En series categóricas, es posible acceder a las propiedades de las categorías a través del atributo `cat`.

In [8]:
# leer las categorias de una serie
serie_jan = pd.Series(log_levels_jan)


In [9]:
# accedemos a las categorias
serie_jan.cat.categories

Index(['debug', 'info', 'warning', 'error', 'critical'], dtype='object')

In [10]:
# accedemos a los codigos de cada categoria
serie_jan.cat.codes

0    0
1    0
2    1
3    1
4    3
5    0
dtype: int8

# Ejercicio

1. Vamos a transformar la columna "internet.protocol" del `DataFrame` `df_events` en una de tipo Categorical, donde las categorías son las siguientes (ordenadas): 
`["TCP", "DNS", "ARP", "DHCP", "ICMP", "MDNS", "NTP", "ICMPv6", "IGMPv3", "MQTT"]`
2. Ordenar el `DataFrame` en primer lugar por "internet.sourceIP", y en segundo órden de prioridad "internet.protocol".

In [11]:
# Completar
from pandas.api.types import CategoricalDtype

categories = ["TCP", "DNS", "ARP", "DHCP", "ICMP", "MDNS", "NTP", "ICMPv6", "IGMPv3", "MQTT"]
cat_type = CategoricalDtype(categories=categories, ordered=True)
df_events["internet.protocol"] = df_events["internet.protocol"].astype(cat_type)

# o bien
#df_events["internet.protocol"] = pd.Categorical(df_events["internet.protocol"], categories=categories, ordered=True)

df_events.sort_values(by=["internet.sourceIP", "internet.protocol"])

,timestamp,packetLength,info,networkInterface.sourceMAC,networkInterface.destinationMAC,internet.sourceIP,internet.destinationIP,internet.protocol,TCP.sourcePort,TCP.destinationPort,TCP.flags,TCP.calculatedWindowSize,UDP.sourcePort,UDP.destinationPort,MQTT.flags,MQTT.message,MQTT.topic,MQTT.messageLength,MQTT.frameCounter
6537,1517690254695551,342,DHCP Discover - Transaction ID 0x66adbe61,08:00:27:71:f1:05,ff:ff:ff:ff:ff:ff,0.0.0.0,255.255.255.255,DHCP,68,67,-1,-1,-1,-1,-1,-1,-1,-1,1
6539,1517690254696277,342,DHCP Request - Transaction ID 0x66adbe61,08:00:27:71:f1:05,ff:ff:ff:ff:ff:ff,0.0.0.0,255.255.255.255,DHCP,68,67,-1,-1,-1,-1,-1,-1,-1,-1,1
9867,1517691203808573,342,DHCP Discover - Transaction ID 0x94a05351,08:00:27:0e:06:81,ff:ff:ff:ff:ff:ff,0.0.0.0,255.255.255.255,DHCP,68,67,-1,-1,-1,-1,-1,-1,-1,-1,1
9869,1517691203809336,342,DHCP Request - Transaction ID 0x94a05351,08:00:27:0e:06:81,ff:ff:ff:ff:ff:ff,0.0.0.0,255.255.255.255,DHCP,68,67,-1,-1,-1,-1,-1,-1,-1,-1,1
11889,1517691775759842,342,DHCP Discover - Transaction ID 0x4db24846,08:00:27:05:7a:7e,ff:ff:ff:ff:ff:ff,0.0.0.0,255.255.255.255,DHCP,68,67,-1,-1,-1,-1,-1,-1,-1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11911,1517691777184097,110,Multicast Listener Report Message v2,08:00:27:05:7a:7e,33:33:00:00:00:16,fe80::e6fe:3566:a5e3:9b6f,ff02::16,ICMPv6,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
11923,1517691777591992,90,Multicast Listener Report Message v2,08:00:27:05:7a:7e,33:33:00:00:00:16,fe80::e6fe:3566:a5e3:9b6f,ff02::16,ICMPv6,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
11924,1517691777600901,62,Router Solicitation,08:00:27:05:7a:7e,33:33:00:00:00:02,fe80::e6fe:3566:a5e3:9b6f,ff02::2,ICMPv6,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
11951,1517691781601812,62,Router Solicitation,08:00:27:05:7a:7e,33:33:00:00:00:02,fe80::e6fe:3566:a5e3:9b6f,ff02::2,ICMPv6,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#000000"></i></font></a></div>
<a id="temporales"></a>

# 2. Datos temporales

_Pandas_ proporciona objetos y una gran cantidad de funcionalidades para el manejo de datos relativos a fechas y horas. 
Una descripción detallada puede encontrarse en la [(documentación)](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#).

Los objetos utilizados en la representación de datos temporales son:

* `Timestamp`. Que representa marcas de tiempo (en concreto, nanosegundos desde el primer instante del 01/01/1970), y que pueden ser convertidas a distintos formatos.

* `DatetimeIndex`. Que permite construir índices con marcas de tiempo (`TimeStamp`).

* `Period`. Representa periodos de tiempo (rangos de tiempo con un inicio y un final).

* `PeriodIndex`. Permite construir índices con periodos de tiempo.

* `Timedelta`. Permite representar intervalos de tiempo. 

A continuación se describen éstos, así como las principales funciones para su manejo.

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#000000"></i></font></a></div>
<a id="temporales-timestamp"></a>

## `pd.Timestamp`

El objeto `Timestamp` se puede crear a partir de un timestamp, de un string con un formato de fecha, o directamente especificando los campos de la fecha (año, mes día, hora, minutos, segundos, milisegundos, microsegundos y nanosegundos).

### A partir de un timestamp

In [12]:
# Calcuamos el timestamp que es ahora (segundos transcurridos desde 1-1-1970)
import time
now = time.time()
now

1682530068.7040517

In [13]:
# Podemos crear la fecha especificando un stamp pero con precisión de nanosegundos!
pd.Timestamp(now * 1e9)

Timestamp('2023-04-26 17:27:48.704051712')

In [14]:
# También podemos especificar la unidad, indicando que son segundos
pd.Timestamp(now, unit="s")

Timestamp('2023-04-26 17:27:48.704051733')

<div class="alert alert-block alert-info">

<i class="fa fa-info-circle" aria-hidden="true"></i> El timestamp se refiere a los segundos desde el 1-1-1970, en la zona horaria UTC (o GMT).

Podemos especificar un timezone al construir el objeto Timestamp a través de tz, e indicando un string. La lista de timezones se puede consultar [aquí](https://en.wikipedia.org/wiki/List_of_tz_database_time_zones) (columna _TZ database name_).

</div>

In [15]:
# El timestamp se refiere a los segundos desde el 1-1-1970, en la zona horaria UTC (o GMT).
display(pd.Timestamp(now, unit="s", tz="UTC"))
display(pd.Timestamp(now, unit="s", tz="Europe/Madrid"))
display(pd.Timestamp(now, unit="s", tz="America/Chicago"))

Timestamp('2023-04-26 17:27:48.704051733+0000', tz='UTC')

Timestamp('2023-04-26 19:27:48.704051733+0200', tz='Europe/Madrid')

Timestamp('2023-04-26 12:27:48.704051733-0500', tz='America/Chicago')

### A partir de una fecha especificando los campos

El orden:

* Campos obligatorios: año, mes día 
* Campos opcionales (por defecto 0): hora, minuto, segundo, microsegundo y nanosegundo (desde la versión 0.23.0)

In [16]:
# Podemos especificar todos los campos
pd.Timestamp(2019, 2, 1, 12, 5, 1, 2, 3)

Timestamp('2019-02-01 12:05:01.000002003')

In [17]:
# O solo los obligatorios
pd.Timestamp(2019, 2, 1)

Timestamp('2019-02-01 00:00:00')

### A partir de una fecha en formato string

Como puede apreciarse en la documentación, _Pandas_ acepta multitud de formatos para especificar el `Timestamp` [(documentación)](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.Timestamp.html).

Un formato muy extendido es el [ISO FORMAT](https://es.wikipedia.org/wiki/ISO_8601). De forma simplificada, el formato que sigue es YYYY/MM/DDTHH:MM:SS. Si termina en la letra Z, quiere decir que la hora está en UTC.

In [18]:
pd.Timestamp("2019/05/16/17:00:00").isoformat()

'2019-05-16T17:00:00'

In [19]:
display(pd.Timestamp("2019-05-16T17:00:00"))
display(pd.Timestamp("2019-05-16T17:00:00Z"))

Timestamp('2019-05-16 17:00:00')

Timestamp('2019-05-16 17:00:00+0000', tz='UTC')

In [20]:
pd.Timestamp("05-14-2020")

Timestamp('2020-05-14 00:00:00')

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#000000"></i></font></a></div>
<a id="temporales-localize"></a>

## Localizando los objetos `timezone`

Si no se especifica la zona horaria, se crean los denominados _naive_ Timestamps. Si se especifica, éstos se llaman _aware_ Timestamps, y llevan un ajuste (desfase horario).

Podemos utilizar el paquete [`pytz`](http://pytz.sourceforge.net/) para transformar un `pd.Timestamp` _naive_ en _aware_ (consciente de su zona horaria). Para ello, y por centralizar en la forma más general, se puede:

1. Construir un objeto de tipo `pytz.timezone`
2. Utilizar la función `localize` del objeto anterior, pasando como argumento el objeto `pd.Timestamp`

_NOTA: En este caso, la hora real no cambia, ya que se ha especificado con esos valores. Lo que cambia es el timestamp interno que representa los segundos desde 1-1-1970 (en UTC)._

In [21]:
# !pip install pytz

In [22]:
import time
import pytz

In [23]:
pd.Timestamp(2020, 3, 20, 17, 2)

Timestamp('2020-03-20 17:02:00')

In [24]:
pytz.timezone("Europe/Madrid")

<DstTzInfo 'Europe/Madrid' LMT-1 day, 23:45:00 STD>

In [25]:
pytz.timezone("Europe/Madrid").localize(pd.Timestamp(2022, 3, 23, 17, 19))

Timestamp('2022-03-23 17:19:00+0100', tz='Europe/Madrid')

In [26]:
print(pd.Timestamp(2022, 3, 23, 17, 19).timestamp())
print(pytz.timezone("Europe/Madrid").localize(pd.Timestamp(2022, 3, 23, 17, 19)).timestamp())

1648055940.0
1648052340.0


In [27]:
pd.Timestamp(now, unit="s")

Timestamp('2023-04-26 17:27:48.704051733')

In [28]:
time.time()

1682530068.8998048

In [29]:
now = time.time()
# El timestamp determina un instante en el tiempo (independiente de la zona horaria)
# Al especificar el tz, lo que hacemos es construirlo localizado, por lo que representa la fecha en el instante
# representado por now, y aplica el desfase con respecto a Europe/Madrid
display(pd.Timestamp(now, unit="s", tz="Europe/Madrid"))

# En este caso, se construye primero el objeto `pd.Timestamp` (naive), y después se localiza.
# Como este objeto tiene una fecha (año, hora, minutos...), éstos no cambian, pero se aplica el desfase,
# por lo que cambia el timestamp interno
display(pytz.timezone("Europe/Madrid").localize(pd.Timestamp(now, unit="s")))

Timestamp('2023-04-26 19:27:48.909859896+0200', tz='Europe/Madrid')

Timestamp('2023-04-26 17:27:48.909859896+0200', tz='Europe/Madrid')

In [30]:
print(now)
print(pd.Timestamp(now, unit="s", tz="Europe/Madrid").timestamp())
print(pytz.timezone("Europe/Madrid").localize(pd.Timestamp(now, unit="s")).timestamp())

1682530068.90986
1682530068.90986
1682522868.90986


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#000000"></i></font></a></div>
<a id="temporales-atributos"></a>

## Atributos y funciones

A veces es útil acceder a los campos del objeto `pd.Timestamp`, por ejemplo, en funciónes de transformación o agregación. Se puede acceder, entre otros, a los siguientes campos o funciones:

- `year`
- `month`
- `day`
- `hour`
- `minute`
- `second`
- `microsecond`
- `nanosecond`
- `tzname()`
- `day_name()`
- `weekday()` (0 es lunes, y 6 es domingo)
- `strftime()` (Permite formatear el objeto como un string. Más información en la [documentación](http://strftime.org/))

In [31]:
# Ejemplo para day_name
pd.Timestamp("2020/03/20").weekday()

4

In [32]:
# Ejemplo para strftime
pd.Timestamp("2019/08/16").strftime("%Y-%b-%dT%H:%M:%SZ")

'2019-Aug-16T00:00:00Z'

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#000000"></i></font></a></div>
<a id="temporales-series"></a>

## Timestamps en series


La función `pd.to_datetime` permite construir objetos de tipo `Timestamp` del mismo modo que el constructor de este tipo de objetos, pero es mucho más potente y flexible. La referencia completa de esta función puede encontrarse en la [(documentación)](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html).

Una de las funcionalidades más utilizadas es la de aplicar esta función a `Series`, construyendo una `Serie` de tipo `Timestamp`.

In [33]:
# "2 June 2013", "Aug 29, 2014", "2015-06-26", "7/12/16", "02/2/2017 10:05AM", 
serie = pd.Series([1557955141, 1557957141, 1558955141])
serie = pd.Series(["01-01-2020", "02-01-2021", "29-02-2022"])

print("Serie de Strings")
print(serie)
print()

serie_dt = pd.to_datetime(serie, format="%d-%m-%Y", errors="coerce") 
print("Serie de elementos datetime")
print(serie_dt)

type(serie_dt[0])

Serie de Strings
0    01-01-2020
1    02-01-2021
2    29-02-2022
dtype: object

Serie de elementos datetime
0   2020-01-01
1   2021-01-02
2          NaT
dtype: datetime64[ns]


pandas._libs.tslibs.timestamps.Timestamp

# Ejercicio

Vamos a crear varias columnas en un `DataFrame` a partir de la columna `timestamp` de `df_events`:

- date: El contenido será un string con el formato `day/month/year`
- day_name: El contenido será el día de la semana

<div class="alert alert-block alert-info">

<i class="fa fa-info-circle" aria-hidden="true"></i> Las series de timpo datetime tienen un atributo, llamado `dt`, que permite vectorizar el acceso a atributos o funciones propias de un elemento individual. Esto resulta de enorme utilidad a la hora de realizar transformaciones, como `serie.dt.month` por ejemplo.

</div>

In [34]:
# Completar
#df_events["date"] = pd.to_datetime(df_events["timestamp"], unit="us", errors="coerce")
#df_events["day_name"] = df_events["date"].apply(pd.Timestamp.day_name)
#df_events.head()

(
    df_events.
    assign(
        date=lambda df: pd.to_datetime(df_events["timestamp"], unit="us").dt.strftime("%d-%m-%Y"),
        day_name=lambda df: pd.to_datetime(df_events["timestamp"], unit="us").dt.day_name()
    )
)

,timestamp,packetLength,info,networkInterface.sourceMAC,networkInterface.destinationMAC,internet.sourceIP,internet.destinationIP,internet.protocol,TCP.sourcePort,TCP.destinationPort,...,TCP.calculatedWindowSize,UDP.sourcePort,UDP.destinationPort,MQTT.flags,MQTT.message,MQTT.topic,MQTT.messageLength,MQTT.frameCounter,date,day_name
0,1517688352799319,100,Publish Message,08:00:27:71:f1:05,08:00:27:53:41:85,192.168.1.7,192.168.1.5,MQTT,52588,1883,...,229,-1,-1,0x30,62.7216410003,fakeSensor2/sound,32,1,03-02-2018,Saturday
1,1517688352837583,66,1883 > 52588 [ACK] Seq=1 Ack=35 Win=227 Len=...,08:00:27:53:41:85,08:00:27:71:f1:05,192.168.1.5,192.168.1.7,TCP,1883,52588,...,227,-1,-1,-1,-1,-1,-1,1,03-02-2018,Saturday
2,1517688352965108,100,Publish Message,08:00:27:0e:06:81,08:00:27:53:41:85,192.168.1.4,192.168.1.5,MQTT,59662,1883,...,229,-1,-1,0x30,46.1680763098,fakeSensor3/sound,32,1,03-02-2018,Saturday
3,1517688352965147,66,1883 > 59662 [ACK] Seq=1 Ack=35 Win=227 Len=...,08:00:27:53:41:85,08:00:27:0e:06:81,192.168.1.5,192.168.1.4,TCP,1883,59662,...,227,-1,-1,-1,-1,-1,-1,1,03-02-2018,Saturday
4,1517688354027486,100,Publish Message,08:00:27:0e:06:81,08:00:27:53:41:85,192.168.1.4,192.168.1.5,MQTT,59662,1883,...,229,-1,-1,0x30,31.8849343769,fakeSensor3/sound,32,1,03-02-2018,Saturday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17035,1517695676698952,66,1883 > 59662 [ACK] Seq=163 Ack=131771 Win=22...,08:00:27:53:41:85,08:00:27:0e:06:81,192.168.1.5,192.168.1.4,TCP,1883,59662,...,227,-1,-1,-1,-1,-1,-1,1,03-02-2018,Saturday
17036,1517695677347149,100,Publish Message,08:00:27:0e:06:81,08:00:27:53:41:85,192.168.1.4,192.168.1.5,MQTT,59662,1883,...,229,-1,-1,0x30,73.2415646154,fakeSensor3/sound,32,1,03-02-2018,Saturday
17037,1517695677347185,66,1883 > 59662 [ACK] Seq=163 Ack=131805 Win=22...,08:00:27:53:41:85,08:00:27:0e:06:81,192.168.1.5,192.168.1.4,TCP,1883,59662,...,227,-1,-1,-1,-1,-1,-1,1,03-02-2018,Saturday
17038,1517695677451076,100,Publish Message,08:00:27:71:f1:05,08:00:27:53:41:85,192.168.1.7,192.168.1.5,MQTT,52588,1883,...,229,-1,-1,0x30,41.4866182673,fakeSensor2/sound,32,1,03-02-2018,Saturday


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#000000"></i></font></a></div>
<a id="temporales-datetimeindex"></a>

## DatetimeIndex

Esta clase implementa un gran número de funcionalidades que permiten la indexación con marcas de tiempo. Es decir, como un índice formado por elementos de tipo `Timestamp`. 

In [35]:
# ejemplo
serie = pd.Series(["Evento 1","Evento 2", "Evento 3", "Evento 4", "Evento 5"], 
                 index=[pd.Timestamp("2019-01-22"), pd.Timestamp("2019-02-05"),
                  pd.Timestamp("2019-02-19"), pd.Timestamp("2019-02-26"),
                  pd.Timestamp("2019-04-7")])
serie

2019-01-22    Evento 1
2019-02-05    Evento 2
2019-02-19    Evento 3
2019-02-26    Evento 4
2019-04-07    Evento 5
dtype: object

In [36]:
serie

2019-01-22    Evento 1
2019-02-05    Evento 2
2019-02-19    Evento 3
2019-02-26    Evento 4
2019-04-07    Evento 5
dtype: object

<div class="alert alert-block alert-info">

<i class="fa fa-info-circle" aria-hidden="true"></i> Una funcionalidad interesante de este tipo de índices es que permite localizar elementos comprendidos en un espacio de tiempo.
</div>


In [37]:
print(serie.loc["2019-02"]) # Imprime las entrada correspondientes a febrero

print()

print(serie.loc["2019-02-05"]) #Imprime todas las entradas correspondientes a 2019

2019-02-05    Evento 2
2019-02-19    Evento 3
2019-02-26    Evento 4
dtype: object

Evento 2


###  Slicing con índices de marcas temporales

`DatetimeIndex` permite hacer _slicing_ (indexación por rangos), igual que otros índices o columnas, pero resulta de especial utilidad en el caso de `DatetimeIndex`, ya que son muy legibles.

En el siguiente ejemplo vamos a acceder a los eventos entre las fechas 2019-02-01 2019-02-20: 

In [38]:
serie.loc["2019-02-01":"2019-02-20"]

2019-02-05    Evento 2
2019-02-19    Evento 3
dtype: object

In [39]:
# Incluso interpreta fechas con otra granularidad
serie.loc["2019-02":"2019-03"]

2019-02-05    Evento 2
2019-02-19    Evento 3
2019-02-26    Evento 4
dtype: object

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#000000"></i></font></a></div>
<a id="temporales-datetimeindex-creacion"></a>

## Creación

Cuando se establece una columna de tipo Datetime a índice, ésta se transforma a `DatetimeIndex`:

Tambén se transforma a `DatetimeIndex` cuando establecemos una columna tipo `Timestamp` a índice:

In [40]:
print(
    type(
        df_events.assign(timestamp=lambda df: pd.to_datetime(df["timestamp"], unit="us"))["timestamp"]
    )
)
print(
    type(
        df_events.assign(timestamp=lambda df: pd.to_datetime(df["timestamp"], unit="us")).set_index("timestamp").index
    )
)

<class 'pandas.core.series.Series'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


### Creación de secuencias: `date_range`

Permite generar índices de marcas temporales entre dos marcas de tiempo, o a partir de una marca de tiempo. En el primer caso permite indicar cuantos periodos se generan. También requiere que se indique la frecuencia, que por defecto es _días_. 

In [41]:
inicio = pd.Timestamp("2019-03-01")
fin = pd.Timestamp("2019-06-26")

print(pd.date_range(inicio, fin, freq="M"))
print(pd.date_range(inicio, periods=6, freq="D"))
print(pd.date_range(inicio, fin, periods=6))
print(pd.date_range(inicio, periods=6))  # por defecto la frecuencia es cada día
print(pd.date_range(inicio, fin, freq="80h20min"))

DatetimeIndex(['2019-03-31', '2019-04-30', '2019-05-31'], dtype='datetime64[ns]', freq='M')
DatetimeIndex(['2019-03-01', '2019-03-02', '2019-03-03', '2019-03-04',
               '2019-03-05', '2019-03-06'],
              dtype='datetime64[ns]', freq='D')
DatetimeIndex(['2019-03-01 00:00:00', '2019-03-24 09:36:00',
               '2019-04-16 19:12:00', '2019-05-10 04:48:00',
               '2019-06-02 14:24:00', '2019-06-26 00:00:00'],
              dtype='datetime64[ns]', freq=None)
DatetimeIndex(['2019-03-01', '2019-03-02', '2019-03-03', '2019-03-04',
               '2019-03-05', '2019-03-06'],
              dtype='datetime64[ns]', freq='D')
DatetimeIndex(['2019-03-01 00:00:00', '2019-03-04 08:20:00',
               '2019-03-07 16:40:00', '2019-03-11 01:00:00',
               '2019-03-14 09:20:00', '2019-03-17 17:40:00',
               '2019-03-21 02:00:00', '2019-03-24 10:20:00',
               '2019-03-27 18:40:00', '2019-03-31 03:00:00',
               '2019-04-03 11:20:00', '2019-

Tambén se transforma a `Datetime` cuando la asignamos a una columna de un `DataFrame`:

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> Existen numerosas posibilidades en relación a la frecuencia. Éstas pueden consultarse [aquí](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases). Por ejemplo, la siguiente porción de código genera marcas de tiempo diarias, pero que solamente corresponden a días laborales.
</div>

# Ejercicios

1. Vamos a asignar la columna `timestamp` como índice. Obtener los eventos entre las fechas "2018-02-03 21:05" y "2018-02-03 21:06".
2. Vamos a crear una nueva columna que es una secuencia de fechas, a incrementos de 1 día, desde ahora (que se puede obtener con `pd.Timestamp.now()`)

In [42]:
# 1

(
    df_events
    .assign(
        timestamp=lambda df: pd.to_datetime(df.timestamp, unit="us", errors="coerce")
    )
    .set_index("timestamp")
    .loc["2018-02-03 21:05":"2018-02-03 21:06"]
)

,packetLength,info,networkInterface.sourceMAC,networkInterface.destinationMAC,internet.sourceIP,internet.destinationIP,internet.protocol,TCP.sourcePort,TCP.destinationPort,TCP.flags,TCP.calculatedWindowSize,UDP.sourcePort,UDP.destinationPort,MQTT.flags,MQTT.message,MQTT.topic,MQTT.messageLength,MQTT.frameCounter
timestamp,,,,,,,,,,,,,,,,,,
2018-02-03 21:05:00.455529,100,Publish Message,08:00:27:71:f1:05,08:00:27:53:41:85,192.168.1.7,192.168.1.5,MQTT,52588,1883,0x018,229,-1,-1,0x30,106.873625693,fakeSensor2/sound,32,1
2018-02-03 21:05:00.455564,66,1883 > 52588 [ACK] Seq=119 Ack=96166 Win=227...,08:00:27:53:41:85,08:00:27:71:f1:05,192.168.1.5,192.168.1.7,TCP,1883,52588,0x010,227,-1,-1,-1,-1,-1,-1,1
2018-02-03 21:05:01.306403,99,Publish Message,08:00:27:0e:06:81,08:00:27:53:41:85,192.168.1.4,192.168.1.5,MQTT,59662,1883,0x018,229,-1,-1,0x30,38.133925483,fakeSensor3/sound,31,1
2018-02-03 21:05:01.306441,66,1883 > 59662 [ACK] Seq=119 Ack=95720 Win=227...,08:00:27:53:41:85,08:00:27:0e:06:81,192.168.1.5,192.168.1.4,TCP,1883,59662,0x010,227,-1,-1,-1,-1,-1,-1,1
2018-02-03 21:05:01.527572,100,Publish Message,08:00:27:71:f1:05,08:00:27:53:41:85,192.168.1.7,192.168.1.5,MQTT,52588,1883,0x018,229,-1,-1,0x30,30.8161040828,fakeSensor2/sound,32,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-02-03 21:05:49.700516,66,1883 > 59662 [ACK] Seq=121 Ack=97110 Win=227...,08:00:27:53:41:85,08:00:27:0e:06:81,192.168.1.5,192.168.1.4,TCP,1883,59662,0x010,227,-1,-1,-1,-1,-1,-1,1
2018-02-03 21:05:49.905619,90,"NTP Version 4, client",08:00:27:53:41:85,52:54:00:12:35:00,192.168.1.5,91.189.94.4,NTP,37443,123,-1,-1,-1,-1,-1,-1,-1,-1,1
2018-02-03 21:05:49.963269,60,Who has 192.168.1.5? Tell 192.168.1.1,52:54:00:12:35:00,ff:ff:ff:ff:ff:ff,RealtekU_12:35:00,Broadcast,ARP,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1


In [43]:
# 2

(
    df_events
    .assign(
        aux=lambda df: pd.date_range(pd.Timestamp.now(), freq="D", periods=df.shape[0])
    )
)

,timestamp,packetLength,info,networkInterface.sourceMAC,networkInterface.destinationMAC,internet.sourceIP,internet.destinationIP,internet.protocol,TCP.sourcePort,TCP.destinationPort,TCP.flags,TCP.calculatedWindowSize,UDP.sourcePort,UDP.destinationPort,MQTT.flags,MQTT.message,MQTT.topic,MQTT.messageLength,MQTT.frameCounter,aux
0,1517688352799319,100,Publish Message,08:00:27:71:f1:05,08:00:27:53:41:85,192.168.1.7,192.168.1.5,MQTT,52588,1883,0x018,229,-1,-1,0x30,62.7216410003,fakeSensor2/sound,32,1,2023-04-26 17:27:49.392831
1,1517688352837583,66,1883 > 52588 [ACK] Seq=1 Ack=35 Win=227 Len=...,08:00:27:53:41:85,08:00:27:71:f1:05,192.168.1.5,192.168.1.7,TCP,1883,52588,0x010,227,-1,-1,-1,-1,-1,-1,1,2023-04-27 17:27:49.392831
2,1517688352965108,100,Publish Message,08:00:27:0e:06:81,08:00:27:53:41:85,192.168.1.4,192.168.1.5,MQTT,59662,1883,0x018,229,-1,-1,0x30,46.1680763098,fakeSensor3/sound,32,1,2023-04-28 17:27:49.392831
3,1517688352965147,66,1883 > 59662 [ACK] Seq=1 Ack=35 Win=227 Len=...,08:00:27:53:41:85,08:00:27:0e:06:81,192.168.1.5,192.168.1.4,TCP,1883,59662,0x010,227,-1,-1,-1,-1,-1,-1,1,2023-04-29 17:27:49.392831
4,1517688354027486,100,Publish Message,08:00:27:0e:06:81,08:00:27:53:41:85,192.168.1.4,192.168.1.5,MQTT,59662,1883,0x018,229,-1,-1,0x30,31.8849343769,fakeSensor3/sound,32,1,2023-04-30 17:27:49.392831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17035,1517695676698952,66,1883 > 59662 [ACK] Seq=163 Ack=131771 Win=22...,08:00:27:53:41:85,08:00:27:0e:06:81,192.168.1.5,192.168.1.4,TCP,1883,59662,0x010,227,-1,-1,-1,-1,-1,-1,1,2069-12-15 17:27:49.392831
17036,1517695677347149,100,Publish Message,08:00:27:0e:06:81,08:00:27:53:41:85,192.168.1.4,192.168.1.5,MQTT,59662,1883,0x018,229,-1,-1,0x30,73.2415646154,fakeSensor3/sound,32,1,2069-12-16 17:27:49.392831
17037,1517695677347185,66,1883 > 59662 [ACK] Seq=163 Ack=131805 Win=22...,08:00:27:53:41:85,08:00:27:0e:06:81,192.168.1.5,192.168.1.4,TCP,1883,59662,0x010,227,-1,-1,-1,-1,-1,-1,1,2069-12-17 17:27:49.392831
17038,1517695677451076,100,Publish Message,08:00:27:71:f1:05,08:00:27:53:41:85,192.168.1.7,192.168.1.5,MQTT,52588,1883,0x018,229,-1,-1,0x30,41.4866182673,fakeSensor2/sound,32,1,2069-12-18 17:27:49.392831


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#000000"></i></font></a></div>
<a id="temporales-period"></a>

## Period

Permite representar un periodo de tiempo. El siguiente código código crea un periodo como el segundo mes de 2016. Lo imprime, e imprime el resultado de varios campos que pueden resultar de utilidad.

In [44]:
periodo = pd.Period("02/2016")
print(periodo)                     # Periodo
print(periodo.days_in_month)       # Días del mes correspondiente al periodo (enero si no se indica nada)
print(periodo.start_time)          # Marca de tiempo en la que comienza el periodo
print(periodo.end_time)            # Marca de tiempo en la que termina el periodo
print(periodo.is_leap_year)        # Si el año en el que está comprendido el periodo es bisiesto

2016-02
29
2016-02-01 00:00:00
2016-02-29 23:59:59.999999999
True


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#000000"></i></font></a></div>
<a id="temporales-PeriodIndex"></a>

## PeriodIndex: Indexación con periodos de tiempo 

<br>
Pandas también implementa índices construidos con periodos de tiempo (no marcas).

In [45]:
serie = pd.Series(["evento1", "evento2", "evento3", "evento4"], 
                [pd.Period("2017/2"), pd.Period("2017/2"), pd.Period("2017/3"), pd.Period("2017/5")])
print(serie)
print()
print(serie.loc["2017-02"])
print(serie.loc["2017"])

2017-02    evento1
2017-02    evento2
2017-03    evento3
2017-05    evento4
Freq: M, dtype: object

2017-02    evento1
2017-02    evento2
Freq: M, dtype: object
2017-02    evento1
2017-02    evento2
2017-03    evento3
2017-05    evento4
Freq: M, dtype: object


### `period_range`

Es similar a `date_range` descrita anteriormente, con las mismas funcionalidades, solo que esta genera secuencias `PeriodIndex`, es decir, índices construidos a partir de periodos de tiempo.

In [46]:
pd.period_range(start="2018-01-15", end="2019-07-22", freq="3M")

PeriodIndex(['2018-01', '2018-04', '2018-07', '2018-10', '2019-01', '2019-04',
             '2019-07'],
            dtype='period[3M]', freq='3M')

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#000000"></i></font></a></div>
<a id="temporales-PeriodIndex"></a>

## TimeDelta: Intervalos de tiempo

Los objetos `TimeDelta`, permiten representar intervalos de tiempo y operar con ellos. Consultar [documentación](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timedelta.html) oficial.

In [47]:
intervalo = pd.Timedelta("1H")
print(intervalo)

intervalo = pd.Timedelta("7D 3H")
print(intervalo)

otro_intervalo = pd.Timedelta("27D 6H 3T")
print(intervalo+otro_intervalo)

0 days 01:00:00
7 days 03:00:00
34 days 09:03:00


Las operaciones entre objetos `Timestamp` generan intervalos de tiempo.

In [48]:
intervalo = pd.Timestamp("Mar 10, 2018") - pd.Timestamp("3 February 2018")
print(intervalo)
print(type(intervalo))

35 days 00:00:00
<class 'pandas._libs.tslibs.timedeltas.Timedelta'>


También es posible operar entre marcas e intervalos de tiempo.

In [49]:
intervalo.total_seconds()

3024000.0

In [50]:
deadline = pd.Timestamp("2020-03-20T20:30") + pd.Timedelta("30d")
print(deadline)
print(type(deadline))

2020-04-19 20:30:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


# Ejercicio

Vamos a calcular el menor timestamp del `DataFrame` df_events. Posteriormente, vamos a crear una nueva columna, llamada `relativeTimestamp`, que será una `Series` de pd.Timedelta, obtenido tras restar la columna `timestamp` con el mínimo timestamp.

In [51]:
# Completar
t = df_events.timestamp - df_events.timestamp.min()
t = pd.to_datetime(t, unit="us", errors="coerce")
# print(t)

min_timestamp = pd.to_datetime(df_events.timestamp, unit="us", errors="coerce").min()

df_events.relativeTimestamp = pd.to_datetime(df_events.timestamp, unit="us", errors="coerce") - min_timestamp
df_events.relativeTimestamp

<ipython-input-51-968e7995c737>:8: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_events.relativeTimestamp = pd.to_datetime(df_events.timestamp, unit="us", errors="coerce") - min_timestamp


0              0 days 00:00:00
1       0 days 00:00:00.038264
2       0 days 00:00:00.165789
3       0 days 00:00:00.165828
4       0 days 00:00:01.228167
                 ...          
17035   0 days 02:02:03.899633
17036   0 days 02:02:04.547830
17037   0 days 02:02:04.547866
17038   0 days 02:02:04.651757
17039   0 days 02:02:04.651775
Name: timestamp, Length: 17040, dtype: timedelta64[ns]

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#000000"></i></font></a></div>
<a id="temporales-resample"></a>

# Resample()

Esta función es muy útil para escalonar la `Serie` o el `DataFrame` cuyo índice es temporal (`pd.DatetimeIndex`, `pd.PeriodIndex` o `pd.TimedeltaIndex`) por frecuencia (por días, horas, minutos, etc.). 

La función puede ser parametrizada con múltiples argumentos ([documentación](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html)), pero el principal, llamado `rule`, es un string que determina la frecuencia sobre la que resamplear ("1D" es 1 día, "3T" es 3 minutos, "1W 3D 5T 2S" es una semana, 3 dias 5 minutos y 2 segundos, ...).

Al aplicar la función, se devuelve un objeto especial de tipo `DatetimeIndexResampler`. Este objeto tiene una función muy parecida a la de `groupby`. La diferencia es, que en este caso, hay dos tipos bien diferenciados de agrupaciones:

- **Downsampling**: Reducción del número de eventos (filas)
- **Upsampling u Oversampling**: Aumento del número de eventos (filas)

Mas información sobre como aplicar funciones para _downsampling/upsamping_ en la [documentación](https://pandas.pydata.org/pandas-docs/stable/reference/resampling.html).

In [52]:
# Creamos una serie con índice temporal que contiene N filas
N = 20
serie_ts = pd.Series([f"evento_{i}" for i in range(N)], index=pd.date_range("2019/02/02", periods=N))

df_ts = pd.DataFrame(dict(eventos=serie_ts, valores=range(N)))

df_ts.head()

,eventos,valores
2019-02-02,evento_0,0
2019-02-03,evento_1,1
2019-02-04,evento_2,2
2019-02-05,evento_3,3
2019-02-06,evento_4,4


## Downsampling o subsampling

Se pueden aplicar las mismas funciones que en el caso de `groupby`, ya que en el fondo la casuística es similar. Dentro de las más importantes destacamos:

- `apply`
- `transform`
- `agg`
- funciones de agregación como `sum`, `mean`, `unique`, `nunique`, `count`, ...

In [53]:
# downsample
df_ts.resample("3D 5T 2S").agg({
    "eventos": ["unique", "count"],
    "valores": ["mean", "count"]
})

eventos       valores  \
                                                       unique count    mean   
2019-02-02 00:00:00  [evento_0, evento_1, evento_2, evento_3]     4     1.5   
2019-02-05 00:05:02            [evento_4, evento_5, evento_6]     3     5.0   
2019-02-08 00:10:04            [evento_7, evento_8, evento_9]     3     8.0   
2019-02-11 00:15:06         [evento_10, evento_11, evento_12]     3    11.0   
2019-02-14 00:20:08         [evento_13, evento_14, evento_15]     3    14.0   
2019-02-17 00:25:10         [evento_16, evento_17, evento_18]     3    17.0   
2019-02-20 00:30:12                               [evento_19]     1    19.0   

                           
                    count  
2019-02-02 00:00:00     4  
2019-02-05 00:05:02     3  
2019-02-08 00:10:04     3  
2019-02-11 00:15:06     3  
2019-02-14 00:20:08     3  
2019-02-17 00:25:10     3  
2019-02-20 00:30:12     1

## Upsampling u Oversampling

Al crear nuevas instancias, estas nuevas posiciones contendrían NaNs. Existen funciones para rellenar estos valores en función de los valores vecinos (o cercanos). Dentro de las más importantes, recalcamos:

- `bfill`
- `ffill`
- `interpolate` (ver [documentación](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.resample.Resampler.interpolate.html#pandas.core.resample.Resampler.interpolate))

In [54]:
df_ts.resample("12H").ffill().head()

,eventos,valores
2019-02-02 00:00:00,evento_0,0
2019-02-02 12:00:00,evento_0,0
2019-02-03 00:00:00,evento_1,1
2019-02-03 12:00:00,evento_1,1
2019-02-04 00:00:00,evento_2,2


In [55]:
df_ts.resample("12H").bfill().head()

,eventos,valores
2019-02-02 00:00:00,evento_0,0
2019-02-02 12:00:00,evento_1,1
2019-02-03 00:00:00,evento_1,1
2019-02-03 12:00:00,evento_2,2
2019-02-04 00:00:00,evento_2,2


# Ejercicios

1. Combina los `DataFrames` df_events y df_predictions, estableciendo previamente a la unión la columna timestamp como índice en format `pd.Timestamp`.
2. Realiza un downsampling por 15 minutos (`15T`) de frecuencia sobre las columnas ["packetLength", "packetLengthPredict"], y agrega computando el valor máximo, mínimo, la media y la desviación estándar. ¿Ves alguna diferencia de los tamaños por tiempo?

In [56]:
# Completar

(
    df_events
    .merge(
        df_predictions,
        on="timestamp"
    )
    .assign(
        timestamp=lambda df:pd.to_datetime(df.timestamp, unit="us")
    )
    .set_index("timestamp")
    .resample("15T")
    .agg({
        "packetLength": ["max", "min", "mean", "std"],
        "packetLengthPredict": ["max", "min", "mean", "std"]
    })
)

packetLength                              \
                             max   min       mean        std   
timestamp                                                      
2018-02-03 20:00:00        590.0  42.0  82.753344  29.865269   
2018-02-03 20:15:00        590.0  42.0  82.716866  30.851567   
2018-02-03 20:30:00        590.0  42.0  84.156051  33.105292   
2018-02-03 20:45:00        590.0  42.0  84.489281  34.259784   
2018-02-03 21:00:00        590.0  42.0  87.560474  44.504121   
2018-02-03 21:15:00          NaN   NaN        NaN        NaN   
2018-02-03 21:30:00          NaN   NaN        NaN        NaN   
2018-02-03 21:45:00        590.0  42.0  83.833333  31.651852   
2018-02-03 22:00:00        590.0  42.0  83.181484  31.460368   

                    packetLengthPredict                                   
                                    max        min       mean        std  
timestamp                                                                 
2018-02-03 20:00:00          590.000017  51.210082  82.770998  29.820763  
2018-02-03 20:15:00          590.000020  51.210013  82.822523  30.941912  
2018-02-03 20:30:00          590.000029  51.210039  84.053533  32.469693  
2018-02-03 20:45:00          590.000029  51.210017  84.239036  33.390866  
2018-02-03 21:00:00          590.000029  51.210077  87.293944  43.434721  
2018-02-03 21:15:00                 NaN        NaN        NaN        NaN  
2018-02-03 21:30:00                 NaN        NaN        NaN        NaN  
2018-02-03 21:45:00          590.000029  51.210029  84.224844  31.928209  
2018-02-03 22:00:00          590.000027  51.210035  83.258067  31.468594